# Tutorial 1: Data preparation for Ghana adaptation example



In [ ]:
#data folder link for MAC
data_folder = "/Users/lena/OneDrive - OnTheHub - The University of Oxford/Work/World Bank/Uruguay/data"
#data folder link for Windows users


In [ ]:
Load libraries

In [ ]:
#Lena to input libraries

## Activity 1: Extract infrastructure data

In [ ]:
basepath = '...'

#### Step 4) Load the road dataset you've just downloaded

In [ ]:
GHA_OSM_roads = gpd.read_file(os.path.join(basepath, 'incoming/openstreetmap/ghana-latest-free/gis_osm_roads_free_1.shp'))

#### Step 5) To take a look at the data and the attribute table fill in and run the next two cells

In [ ]:
GHA_OSM_roads.

#### Step 6) Next we want to make a couple of changes to the data

In [ ]:
GHA_OSM_roads = GHA_OSM_roads.to_crs("EPSG:32630")

In [ ]:
GHA_OSM_roads = GHA_OSM_roads[['','']]
GHA_OSM_roads = GHA_OSM_roads.reset_index()
GHA_OSM_roads = GHA_OSM_roads.rename(columns = {'':''})

In [ ]:
GHA_OSM_roads.head()

#### Step 7) Save the pre-processed dataset

## Activity 2: Extract hazard data

## Activity 3: Intersect hazard ...